In [1]:
import os
import time
import librosa
import zipfile
import numpy as np
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [6]:
!wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O urban8k.tgz

import shutil
import os

dataset_path = '/content/gdrive/MyDrive/urbansound8k'
shutil.copy('urban8k.tgz', os.path.join(dataset_path, 'urban8k.tgz'))

!tar -xzf urban8k.tgz
!rm urban8k.tgz

In [4]:
import shutil
import os

dataset_path = '/content/gdrive/MyDrive/urbansound8k'
shutil.copy(os.path.join(dataset_path, 'urban8k.tgz'), 'urban8k.tgz')

!tar -xzf urban8k.tgz
!rm urban8k.tgz

[Errno 2] No such file or directory: 'urbansound8k.tgz'
tar (child): urbansound8k.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
rm: cannot remove 'urbansound8k.tgz': No such file or directory


In [7]:
# FeatureExtractor class including librosa audio processing functions
class FeatureExtractor:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.max_audio_duration = 4
        self.dataset_df = self._create_dataset(csv_file)

    @staticmethod
    def _create_dataset(csv_file):
        """
        Args:
            dataset_path: path with the .wav files after unzipping
        Returns: A pandas dataframe with the list of files and labels (`filenames`, `labels`)
        """
        dataset_df = pd.read_csv(csv_file)
        filepaths = []
        for i, row in dataset_df.iterrows():
            filepaths.append(os.path.join('UrbanSound8K/audio', 'fold'+str(row['fold']), row['slice_file_name']))
        dataset_df['filepath'] = filepaths
        return dataset_df

    @staticmethod
    def _compute_max_pad_length(max_audio_length, sample_rate=22050, n_fft=2048, hop_length=512):
        dummy_file = np.random.random(max_audio_length*sample_rate)
        stft = librosa.stft(dummy_file, n_fft=n_fft, hop_length=hop_length)
        # Return an even number for CNN computation purposes
        if stft.shape[1] % 2 != 0:
            return stft.shape[1] + 1
        return stft.shape[1]

    def compute_save_features(self,
                        mode='mfcc',
                        sample_rate=22050,
                        n_fft=2048,
                        hop_length=512,
                        n_mfcc=40,
                        output_path='features',
                        deltas=False
                        ):
        dataset_features = []
        max_pad = self._compute_max_pad_length(self.max_audio_duration,
                                               sample_rate=sample_rate,
                                               n_fft=n_fft,
                                               hop_length=hop_length)
        print('Max Padding = ', max_pad)

        if not os.path.exists(output_path):
            print('Creating output folder: ', output_path)
            os.makedirs(output_path)
        else:
            print('Output folder already existed')

        print('Saving features in ', output_path)
        i = 0
        t = time.time()

        features_path = []
        for filepath in self.dataset_df['filepath']:
            if i % 100 == 0:
                print('{} files processed in {}s'.format(i, time.time() - t))
            audio_file, sample_rate = librosa.load(filepath, sr=sample_rate, res_type='kaiser_fast')
            if mode == 'mfcc':
                audio_features = self.compute_mfcc(audio_file, sample_rate, n_fft, hop_length, n_mfcc, deltas)
            elif mode == 'stft':
                audio_features = self.compute_stft(audio_file, sample_rate, n_fft, hop_length)
            elif mode == 'mel-spectogram':
                audio_features = self.compute_mel_spectogram(audio_file, sample_rate, n_fft, hop_length)

            audio_features = np.pad(audio_features,
                                    pad_width=((0, 0), (0, max_pad - audio_features.shape[1])))

            save_path = os.path.join(output_path, filepath.split('/')[-1].replace('wav', 'npy'))
            self.save_features(audio_features, save_path)
            features_path.append(save_path)
            i+=1
        self.dataset_df['features_path'] = features_path
        return self.dataset_df

    @staticmethod
    def save_features(audio_features, filepath):
        np.save(filepath, audio_features)

    @staticmethod
    def compute_mel_spectogram(audio_file, sample_rate, n_fft, hop_length):
        return librosa.feature.melspectrogram(y=audio_file,
                                              sr=sample_rate,
                                              n_fft=n_fft,
                                              hop_length=hop_length)
    @staticmethod
    def compute_stft(audio_file, sample_rate, n_fft, hop_length):
        return librosa.stft(y=audio_file, n_fft=n_fft, hop_length=hop_length)

    @staticmethod
    def compute_mfcc(audio_file, sample_rate, n_fft, hop_length, n_mfcc, deltas=False):
        mfccs = librosa.feature.mfcc(y=audio_file,
                                    sr=sample_rate,
                                    n_fft=n_fft,
                                    n_mfcc=n_mfcc,
                                    )
        # Change mode from interpolation to nearest
        if deltas:
          delta_mfccs = librosa.feature.delta(mfccs, mode='nearest')
          delta2_mfccs = librosa.feature.delta(mfccs, order=2, mode='nearest')
          return np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
        return mfccs

In [8]:
# Create dataset and extract features
fe = FeatureExtractor('UrbanSound8K/metadata/UrbanSound8K.csv')

In [10]:
!pip install resampy
!pip install samplerate
import resampy
import samplerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 11.5 MB/s eta 0:00:00


Access to disc and librosa loading of audio files is very slow on colab Notebook (30-40 min) we could load the pre-computed features instead.

In [13]:
# Uncomment and run to compute and save features on the colab notebook
dataset_df = fe.compute_save_features(mode='mfcc', n_mfcc=13, output_path='features_mfcc', deltas=True)

Max Padding =  174
Output folder already existed
Saving features in  features_mfcc
0 files processed in 0.0002880096435546875s


ModuleNotFoundError: ignored

In [ ]:
dataset_df['features'] = [np.asarray(np.load(feature_path)) for feature_path in dataset_df['features_path']]

In [ ]:
from keras.utils import to_categorical
dataset_df['labels_categorical'] = [to_categorical(label, 10) for label in dataset_df['classID']]

In [ ]:
dataset_df.head()

import pickle
with open('dataset_df.pickle','wb') as f:
     pickle.dump(dataset_df, f)


In [ ]:
shutil.copy('dataset_df.pickle', '/content/gdrive/MyDrive/urbansound8k')

'/content/gdrive/MyDrive/urbansound8k/dataset_df.pickle'